In [187]:
import psycopg2 as pg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

from config import DATABASE_URI

idx = pd.IndexSlice

In [188]:
test = "ABBC"

In [189]:
sql = """
SELECT
    date,
    closeadj
FROM
    prices
WHERE
    ticker = 'SPY';
"""

with pg.connect(DATABASE_URI) as conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        results = cur.fetchall()

market = pd.Series(dict(results), dtype="float").sort_index()
market = market.pct_change().dropna()

In [190]:
sql = f"""
SELECT
    date,
    closeadj,
    close,
    volume
FROM
    prices
WHERE
    ticker = '{test}';
"""

with pg.connect(DATABASE_URI) as conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        results = cur.fetchall()
        
prices_df = pd.DataFrame.from_records(results, columns=["date", "closeadj", "close", "volume"], coerce_float=True)
prices_df = prices_df.set_index("date", verify_integrity=True).sort_index()
# Sometimes columns are totally empty so pandas fills with None but we actually want NaN
# for math
prices_df = prices_df.replace([None], np.nan)

In [191]:
sql = f"""
SELECT
    datekey,
    sharesbas,
    pb,
    assetsc,
    cashneq,
    liabilitiesc,
    depamor,
    roa,
    assets,
    divyield,
    debt,
    revenue
FROM
    fundamentals
WHERE
    (
        ticker = '{test}'
        AND dimension = 'ART'
    );
"""

with pg.connect(DATABASE_URI) as conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        results = cur.fetchall()

fund_df = pd.DataFrame.from_records(results, columns=["date", "shares", "pb",
                                                      "assetsc", "cash", "liabilitiesc",
                                                      "deprec", "roa", "assets", "divyield",
                                                      "debt", "revenue"],
                                    coerce_float=True)
fund_df["date"] = pd.to_datetime(fund_df["date"])
fund_df = fund_df.set_index("date", verify_integrity=True).sort_index()
fund_df = fund_df.replace([None], np.nan)

In [192]:
df = fund_df.reindex(index=prices_df.index, method="ffill", limit=252)
df = df.join(prices_df)

In [193]:
dates = pd.bdate_range(start=df.index[0], end=df.index[-1], freq='M')

In [194]:
rolling_dfs = []

for date in dates:
    annual_df = df.loc[(date - pd.Timedelta(days=365)):date]
    if len(annual_df) > 250:
        rolling_dfs.append(annual_df)

In [195]:
def calc_beta(stock_df, market_df):
    """
    Calculates beta from daily stock returns.
    
    Inputs:
    stock_df: stock returns
    market_df: market index RETURNS
    """
    # Try to get market returns for same days as stock returns, if they don't match up, catch index KeyError
    # and return NaN
    try:
        market_df = market_df.loc[stock_df.index]
    except KeyError:
        print("Market returns do not exist for some days in stock data. Skipping")
        return np.nan
    beta = np.sum((stock_df - np.mean(stock_df)) * (market_df - np.mean(market_df))) / np.sum((market_df - np.mean(market_df))**2)
    return beta

In [196]:
def make_features(df, market_df):
    result = {"date": df.index[-1]}
    # log of market capitalization
    result["logsize"] = np.log(df.iloc[-1]["close"] * df.iloc[-1]["shares"])
    # price / book ratio
    result["pb"] = df.iloc[-1]["pb"]
    # -12 month to -2 month stock return
    result["momentum"] = (df.iloc[-1]["closeadj"] / df.iloc[-43]["closeadj"]) - 1
    # log growth in outstanding shares
    result["issuance"] = np.log(df.iloc[-1]["shares"]) - np.log(df.iloc[0]["shares"])
    # log growth in non-cash working capital minus depreciation
    result["accruals"] = ((df.iloc[-1]["assetsc"] - df.iloc[-1]["cash"] - df.iloc[-1]["liabilitiesc"]) / (df.iloc[0]["assetsc"] - df.iloc[0]["cash"] - df.iloc[0]["liabilitiesc"])) - 1
    # return on assets
    result["roa"] = df.iloc[-1]["roa"]
    # log growth in assets
    result["assets"] = np.log(df.iloc[-1]["assets"]) - np.log(df.iloc[0]["assets"])
    # dividend yield
    result["divyield"] = df.iloc[-1]["divyield"]
    # beta
    returns = df["closeadj"].pct_change().dropna()
    result["beta"] = calc_beta(returns, market_df)
    # standard deviation of daily returns
    result["stddev"] = np.std(returns)
    # average daily share turnover
    result["turnover"] = np.mean(df["volume"] / df["shares"])
    # debt / market cap
    result["debt_price"] = df.iloc[-1]["debt"] / (df.iloc[-1]["close"] * df.iloc[-1]["shares"])
    # sales / market cap
    result["sales_price"] = df.iloc[-1]["revenue"] / (df.iloc[-1]["close"] * df.iloc[-1]["shares"])
    result["monthly_ret"] = (df.iloc[-1]["closeadj"] / df.iloc[-22]["closeadj"]) - 1
    return result

In [197]:
final_df = pd.DataFrame([make_features(n, market) for n in rolling_dfs])
final_df["forward_ret"] = final_df["monthly_ret"].shift(-1)
final_df = final_df.dropna()